In [1]:
import urllib
import urllib.request
import http
import http.cookiejar
import lxml
from lxml import etree
import MySQLdb
import time
import re

In [2]:
def get_page_range():
    #查找网页总量
    #get Proxy in https://www.xicidaili.com/nn/
    proxy_url = 'https://www.xicidaili.com/nt/'
    opener = urllib.request.build_opener()
    headers = [('Referer',proxy_url),\
               ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
               ('Cache-Control','max-age=0'),\
               ('Content-Type','application/x-www-form-urlencoded'),\
               ('Origin',proxy_url),\
               ('Connection','keep-alive'),\
               ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
               ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
    opener.addheaders = headers
    request = opener.open(proxy_url)
    request_read = request.read()
    req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
    page_range = req_xml.xpath('//*[@id="body"]/div[3]/a[{}]/text()'.format(len(req_xml.xpath('//*[@id="body"]/div[3]/*'))-4))[0]
    print(page_range)
    opener.close()
    return int(page_range)

In [3]:
def get_proxy_info(proxy_url):
    #获取代理信息
    #proxy_url='https://www.xicidaili.com/nn/3'
    opener = urllib.request.build_opener()
    headers = [('Referer',proxy_url),\
               ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
               ('Cache-Control','max-age=0'),\
               ('Content-Type','application/x-www-form-urlencoded'),\
               ('Origin',proxy_url),\
               ('Connection','keep-alive'),\
               ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
               ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
    opener.addheaders = headers
    #proxy_handler = urllib.request.ProxyHandler({'https':'{}:{}'.format('112.85.165.126',9999),'http':'{}:{}'.format('112.85.165.126',9999)})
    #opener.add_handler(proxy_handler)
    try:
        request = opener.open(proxy_url,timeout=10)
        request_read = request.read()
    except Exception as ex:
        request_read = bytes('Exception:{}'.format(str(ex)),encoding='UTF8')
        print(request_read.decode(encoding='UTF8',errors='ignore'))
        return 0
    req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
    proxy_info_all = []
    for proxy_num in range(2,101):
        proxy_ip=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[2]/text()'.format(proxy_num))[0]))
        proxy_port=(int(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[3]/text()'.format(proxy_num))[0]))
        try:
            proxy_location=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[4]/a/text()'.format(proxy_num))[0]))
        except Exception as ex:
            proxy_location='unknown'
        proxy_anon=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[5]/text()'.format(proxy_num))[0]))
        proxy_type=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[6]/text()'.format(proxy_num))[0]))
        proxy_speed=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[7]/div//@title'.format(proxy_num))[0]))
        proxy_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[8]/div//@title'.format(proxy_num))[0]))
        proxy_life=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[9]/text()'.format(proxy_num))[0]))
        proxy_web_check_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[10]/text()'.format(proxy_num))[0]))
        proxy_info = [proxy_ip,proxy_port,proxy_location,proxy_anon,proxy_type,proxy_speed,proxy_time,proxy_life,proxy_web_check_time]
        proxy_info_all.append(proxy_info)
    print(proxy_info_all)
    opener.close()
    proxy_to_db(proxy_info_all)    

In [4]:
def proxy_to_db(proxy_info_all):
    proxy_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8' )
    proxy_cursor = proxy_db.cursor()
    sql_stmt = "insert into t_proxy_info(proxy_ip,proxy_port,proxy_location,proxy_anon,proxy_type,proxy_speed,proxy_time,proxy_life,proxy_web_check_time)\
            select %s,%s,%s,%s,%s,%s,%s,%s,%s from dual\
            where NOT EXISTS (select 1 from t_proxy_info where proxy_ip = %s and proxy_port = %s)"
    for i in range(len(proxy_info_all)):
        #print(tuple(proxy_info_all[i]+proxy_info_all[i][0:2]))
        proxy_cursor.execute(sql_stmt,tuple(proxy_info_all[i]+proxy_info_all[i][0:2]))
    proxy_db.commit()
    proxy_db.close()

In [5]:
def get_proxy():
    #page_range=get_page_range()
    page_range=1000
    print(page_range)
    for n in range(3,page_range):
        proxy_url = 'https://www.xicidaili.com/nt/{}'.format(n)
        time.sleep(600)
        print(proxy_url)
        get_proxy_info(proxy_url)

In [6]:
def check_proxy():
    proxy_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8')
    proxy_cursor = proxy_db.cursor()
    sql_stmt = "SELECT * FROM NTTask.t_proxy_info"
    proxy_cursor.execute(sql_stmt)
    proxy_list_all = proxy_cursor.fetchall()
    for i in range(len(proxy_list_all)):
        print(proxy_list_all[i])
        check_status = check_proxy_status(proxy_list_all[i])
        print(check_status)
        #插入检查记录
        sql_stmt = 'insert into t_proxy_check_log\
                    values(%s,%s,%s,sysdate())'
        print(list(proxy_list_all[i][0:2])+[check_status[1]])        
        proxy_cursor.execute(sql_stmt,tuple(list(proxy_list_all[i][0:2])+[check_status[1]]))
        
        #更新整体记录表
        if check_status[0] == 1:
            sql_stmt = 'update t_proxy_info a set a.proxy_last_check_time=sysdate(),a.proxy_last_check_status=%s,\
                        a.proxy_all_check_times=ifnull(a.proxy_all_check_times,0)+1,a.proxy_succ_check_times=ifnull(a.proxy_succ_check_times,0)+1\
                        where a.proxy_ip=%s and a.proxy_port=%s'
            proxy_cursor.execute(sql_stmt,tuple([check_status[1]]+list(proxy_list_all[i][0:2])))
        else:
            sql_stmt = 'update t_proxy_info a set a.proxy_last_check_time=sysdate(),a.proxy_last_check_status=%s,\
                        a.proxy_all_check_times=ifnull(a.proxy_all_check_times,0)+1\
                        where a.proxy_ip=%s and a.proxy_port=%s'
            proxy_cursor.execute(sql_stmt,tuple([check_status[1]]+list(proxy_list_all[i][0:2])))
        proxy_db.commit()
    proxy_cursor.close()
    proxy_db.close()

In [7]:
def check_proxy_status(proxy_info):
    if proxy_info[4] == 'HTTPS':
        check_url = 'https://ip.cn/'
    else:
        check_url = 'http://200019.ip138.com/'

    opener = urllib.request.build_opener()
    #添加headers
    headers = [('Referer',check_url),\
               ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
               ('Cache-Control','max-age=0'),\
               ('Content-Type','application/x-www-form-urlencoded'),\
               ('Origin',check_url),\
               ('Connection','keep-alive'),\
               ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
               ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
    opener.addheaders = headers
    proxy_handler = urllib.request.ProxyHandler({'https':'{}:{}'.format(proxy_info[0],proxy_info[1]),'http':'{}:{}'.format(proxy_info[0],proxy_info[1])})
    opener.add_handler(proxy_handler)
    try:
        request = opener.open(check_url,timeout=10)
        request_read = request.read()
        if proxy_info[4] == 'HTTPS':
            req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
            str(req_xml.xpath('//script[1]/text()')[0])
            searchObj = re.search("html\('(.*)'\)",str(req_xml.xpath('//script[1]/text()')[0]), re.M|re.I)
            parse_xml = lxml.etree.HTML(searchObj.group(1))
            check_status = [1,'Success: {}'.format(str(parse_xml.xpath('//div/p/code/text()')))]
        else:
            req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
            check_status = [1,'Success: {}'.format(str(req_xml.xpath('/html/body/p[1]/text()')))]
    except Exception as ex:
        #print('Exception: '+str(ex))
        request_read = bytes('Exception:{}'.format(str(ex)),encoding='UTF8')
        check_status = [0,request_read.decode(encoding='UTF8',errors='ignore')]
    #print(proxy_info[0:2],request_read.decode(encoding='UTF8',errors='ignore'))
    opener.close()
    return check_status

In [ ]:
if __name__ == '__main__':
    get_proxy()
    #check_proxy()

In [ ]:
while True:
    check_proxy()

('182.34.20.220', 9999, '山东烟台', '高匿', 'HTTP', '0.109秒', '0.021秒', '1分钟', '19-07-26 10:20', None, None, None, None)
[0, 'Exception:<urlopen error timed out>']
['182.34.20.220', 9999, 'Exception:<urlopen error timed out>']
('1.197.204.7', 9999, '河南济源', '高匿', 'HTTP', '0.089秒', '0.017秒', '1分钟', '19-07-26 10:20', None, None, None, None)
[0, 'Exception:<urlopen error [Errno 111] Connection refused>']
['1.197.204.7', 9999, 'Exception:<urlopen error [Errno 111] Connection refused>']
('113.121.22.254', 9999, '山东', '高匿', 'HTTP', '0.268秒', '0.053秒', '5天', '19-07-26 10:20', None, None, None, None)
[0, 'Exception:<urlopen error [Errno 111] Connection refused>']
['113.121.22.254', 9999, 'Exception:<urlopen error [Errno 111] Connection refused>']
('120.83.105.25', 9999, '广东揭阳', '高匿', 'HTTP', '0.189秒', '0.037秒', '76天', '19-07-26 10:20', None, None, None, None)
[0, 'Exception:<urlopen error timed out>']
['120.83.105.25', 9999, 'Exception:<urlopen error timed out>']
('120.83.100.252', 9999, '广东揭阳市普宁', '

[0, 'Exception:<urlopen error timed out>']
['115.219.35.38', 8118, 'Exception:<urlopen error timed out>']
('113.121.21.16', 9999, '山东', '高匿', 'HTTP', '0.114秒', '0.022秒', '76天', '19-07-26 06:20', None, None, None, None)
[0, 'Exception:<urlopen error [Errno 111] Connection refused>']
['113.121.21.16', 9999, 'Exception:<urlopen error [Errno 111] Connection refused>']
('113.121.20.42', 9999, '山东烟台市莱阳', '高匿', 'HTTP', '0.111秒', '0.022秒', '1分钟', '19-07-26 06:20', None, None, None, None)
[0, 'Exception:<urlopen error timed out>']
['113.121.20.42', 9999, 'Exception:<urlopen error timed out>']
('112.85.168.130', 9999, '江苏南通', '高匿', 'HTTPS', '0.156秒', '0.031秒', '55天', '19-07-26 06:20', None, None, None, None)
[0, 'Exception:<urlopen error timed out>']
['112.85.168.130', 9999, 'Exception:<urlopen error timed out>']
('113.121.23.57', 9999, '山东', '高匿', 'HTTP', '1.133秒', '0.226秒', '2天', '19-07-26 06:20', None, None, None, None)
[0, 'Exception:<urlopen error timed out>']
['113.121.23.57', 9999, 'Excep

[0, 'Exception:<urlopen error timed out>']
['119.134.110.240', 8118, 'Exception:<urlopen error timed out>']
('1.199.31.234', 9999, '河南漯河', '高匿', 'HTTP', '1.211秒', '0.242秒', '1分钟', '19-07-26 01:20', None, None, None, None)
[0, 'Exception:<urlopen error timed out>']
['1.199.31.234', 9999, 'Exception:<urlopen error timed out>']
('115.207.38.37', 61234, '浙江衢州', '高匿', 'HTTP', '0.211秒', '0.042秒', '1小时', '19-07-26 01:20', None, None, None, None)
[0, 'Exception:[Errno 104] Connection reset by peer']
['115.207.38.37', 61234, 'Exception:[Errno 104] Connection reset by peer']
('36.248.133.93', 9999, '福建福州', '高匿', 'HTTP', '0.22秒', '0.044秒', '40分钟', '19-07-26 01:00', None, None, None, None)
[0, 'Exception:<urlopen error timed out>']
['36.248.133.93', 9999, 'Exception:<urlopen error timed out>']
('121.61.25.201', 61234, '湖北咸宁', '高匿', 'HTTP', '0.14秒', '0.028秒', '3天', '19-07-26 01:00', None, None, None, None)
[0, 'Exception:[Errno 104] Connection reset by peer']
['121.61.25.201', 61234, 'Exception:[Er

[0, 'Exception:<urlopen error [Errno 111] Connection refused>']
['123.169.35.55', 9999, 'Exception:<urlopen error [Errno 111] Connection refused>']
('1.197.16.83', 9999, '河南许昌', '高匿', 'HTTPS', '5.029秒', '1.005秒', '37天', '19-07-25 21:20', None, None, None, None)
[0, 'Exception:<urlopen error timed out>']
['1.197.16.83', 9999, 'Exception:<urlopen error timed out>']
('120.83.106.144', 9999, '广东揭阳', '高匿', 'HTTP', '0.209秒', '0.041秒', '61天', '19-07-25 21:20', None, None, None, None)
[0, 'Exception:<urlopen error [Errno 111] Connection refused>']
['120.83.106.144', 9999, 'Exception:<urlopen error [Errno 111] Connection refused>']
('113.121.22.144', 9999, '山东', '高匿', 'HTTP', '0.105秒', '0.021秒', '1分钟', '19-07-25 21:20', None, None, None, None)
[0, 'Exception:<urlopen error [Errno 111] Connection refused>']
['113.121.22.144', 9999, 'Exception:<urlopen error [Errno 111] Connection refused>']
('120.83.109.50', 9999, '广东揭阳市普宁', '高匿', 'HTTP', '0.244秒', '0.048秒', '1分钟', '19-07-25 21:20', None, None, 

In [ ]:
proxy_info = ('221.178.146.113', 61399, '广东深圳', '高匿', 'HTTPS', '2.524秒', '0.504秒', '1分钟', '19-06-25 13:21', None, None)
if proxy_info[4] == 'HTTPS':
    check_url = 'https://ip.cn/'
else:
    check_url = 'http://200019.ip138.com/'
    
opener = urllib.request.build_opener()
#添加headers
headers = [('Referer',check_url),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',check_url),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers
proxy_handler = urllib.request.ProxyHandler({'https':'{}:{}'.format(proxy_info[0],proxy_info[1]),'http':'{}:{}'.format(proxy_info[0],proxy_info[1])})
opener.add_handler(proxy_handler)
try:
    request = opener.open(check_url,timeout=10)
    request_read = request.read()
    req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
    str(req_xml.xpath('//script[1]/text()')[0])
    searchObj = re.search("html\('(.*)'\)",str(req_xml.xpath('//script[1]/text()')[0]), re.M|re.I)
    print(searchObj.group(1))
    parse_xml = lxml.etree.HTML(searchObj.group(1))
    print('Success: {}'.format(str(parse_xml.xpath('//div/p/code/text()'))))
except Exception as ex:
    #print('Exception: '+str(ex))
    request_read = bytes('Exception:{}'.format(str(ex)),encoding='UTF8')
print(proxy_info[0:2],request_read.decode(encoding='UTF8',errors='ignore'))
opener.close()

In [ ]:
print(str(req_xml.xpath('//script[1]/text()')[0]))

In [ ]:
check_proxy()

In [ ]:
opener = urllib.request.build_opener()
proxy_url='https://www.xicidaili.com/nn/1'
headers = [('Referer',proxy_url),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',proxy_url),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers
print(proxy_url)
request = opener.open(proxy_url)
request_read = request.read()
#print(request_read.decode(encoding='UTF8',errors='ignore'))
req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))

In [ ]:
#print(str(req_xml.xpath('//*[@id="ip_list"]/tr[101]/td[1]/text()')[0]))
for proxy_num in range(2,101):
    proxy_ip=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[2]/text()'.format(proxy_num))[0]))
    proxy_port=(int(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[3]/text()'.format(proxy_num))[0]))
    try:
        proxy_location=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[4]/a/text()'.format(proxy_num))[0]))
    except Exception as ex:
        proxy_location='unknown'
    proxy_anon=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[5]/text()'.format(proxy_num))[0]))
    proxy_type=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[6]/text()'.format(proxy_num))[0]))
    proxy_speed=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[7]/div//@title'.format(proxy_num))[0]))
    proxy_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[8]/div//@title'.format(proxy_num))[0]))
    proxy_life=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[9]/text()'.format(proxy_num))[0]))
    proxy_web_check_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[10]/text()'.format(proxy_num))[0]))
    proxy_info = [proxy_ip,proxy_port,proxy_location,proxy_anon,proxy_type,proxy_speed,proxy_time,proxy_life,proxy_web_check_time]
    print(proxy_info)

In [ ]:
test_url = 'http://200019.ip138.com/'
test_https = 'https://ip.cn/'
opener = urllib.request.build_opener()
#添加headers
headers = [('Referer',proxy_url),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',proxy_url),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers

#添加cookie
#cookie_filename = 'cookie_xicidaili.txt'
#cookies = http.cookiejar.CookieJar()
#cookies = http.cookiejar.MozillaCookieJar(cookie_filename)
#cookies.load(ignore_discard=True, ignore_expires=True)
#cookie_handler = urllib.request.HTTPCookieProcessor(cookies)
#opener.add_handler(cookie_handler)
#添加代理
#proxy_handler = urllib.request.ProxyHandler({'https':'221.178.146.113:61399','http':'221.178.146.113:61399'})
proxy_handler = urllib.request.ProxyHandler({'https':'222.89.32.154:9999','http':'222.89.32.154:9999'})
opener.add_handler(proxy_handler)

#添加PostData
#data = {"jumpurl":"http%3A%2F%2Fwww.itokoo.com%2Findex.php","step":"2","ajax":"1","pwuser":"Flashield","pwpwd":"Zb122525","lgt":"0"}
#post_data = urllib.parse.urlencode(data).encode(encoding='UTF8')


In [ ]:
try:
    request = opener.open(test_https,timeout=10)
    request_read = request.read()
    req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
    req_xml.xpath('//*[@id="result"]/div/p[1]/code/text()')
    print(req_xml.xpath('//*[@id="result"]/div/p[1]/code/text()')[0])
except Exception as ex:
        print('Exception:{}'.format(str(ex)))

In [ ]:
request = opener.open(proxy_url,ti)
request_read = request.read()
req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
proxy_ip = req_xml.xpath('//*[@id="ip_list"]/tr[100]/td[2]/text()')[0]
proxy_port = req_xml.xpath('//*[@id="ip_list"]/tr[100]/td[3]/text()')[0]
proxy_type = req_xml.xpath('//*[@id="ip_list"]/tr[100]/td[6]/text()')[0]
print(proxy_ip,proxy_port,proxy_type)
proxy_handler = urllib.request.ProxyHandler({proxy_type:'{}:{}'.format(proxy_ip, proxy_port)})
opener1 = urllib.request.build_opener()
opener1.add_handler(proxy_handler)

In [ ]:
request = opener1.open(test_url)
request_read = request.read()
request_read.decode(encoding='UTF8',errors='ignore')

In [ ]:
with open('Proxy.html','w') as f:
    f.write(request_read.decode(encoding='GB2312',errors='ignore'))

In [ ]:
req_xml.xpath('//*[@id="result"]/div/p[1]/code/text()')

In [ ]:
req_xml.xpath('//*[@id="body"]/div[2]/a[{}]/text()'.format(len(req_xml.xpath('//*[@id="body"]/div[2]/*'))-4))

In [ ]:
proxy_url='https://www.xicidaili.com/nn/6'
opener = urllib.request.build_opener()
headers = [('Referer',proxy_url),\
           ('Accept-Language','en,zh-CN;q=0.8,zh;q=0.6,en-US;q=0.4'),\
           ('Cache-Control','max-age=0'),\
           ('Content-Type','application/x-www-form-urlencoded'),\
           ('Origin',proxy_url),\
           ('Connection','keep-alive'),\
           ('User-Agent','Mozilla/5.0 (Windows NT 5.2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36'),\
           ('Accept','text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8')]
opener.addheaders = headers
proxy_handler = urllib.request.ProxyHandler({'https':'{}:{}'.format('163.204.246.41',9999),'http':'{}:{}'.format('163.204.246.41',9999)})
opener.add_handler(proxy_handler)
try:
    request = opener.open(proxy_url,timeout=10)
    request_read = request.read()
except Exception as ex:
    request_read = bytes('Exception:{}'.format(str(ex)),encoding='UTF8')
print(request_read.decode(encoding='UTF8',errors='ignore'))
#return 0
req_xml = lxml.etree.HTML(request_read.decode(encoding='UTF8',errors='ignore'))
proxy_info_all = []
for proxy_num in range(2,101):
    proxy_ip=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[2]/text()'.format(proxy_num))[0]))
    proxy_port=(int(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[3]/text()'.format(proxy_num))[0]))
    try:
        proxy_location=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[4]/a/text()'.format(proxy_num))[0]))
    except Exception as ex:
        proxy_location='unknown'
    proxy_anon=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[5]/text()'.format(proxy_num))[0]))
    proxy_type=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[6]/text()'.format(proxy_num))[0]))
    proxy_speed=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[7]/div//@title'.format(proxy_num))[0]))
    proxy_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[8]/div//@title'.format(proxy_num))[0]))
    proxy_life=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[9]/text()'.format(proxy_num))[0]))
    proxy_web_check_time=(str(req_xml.xpath('//*[@id="ip_list"]/tr[{}]/td[10]/text()'.format(proxy_num))[0]))
    proxy_info = [proxy_ip,proxy_port,proxy_location,proxy_anon,proxy_type,proxy_speed,proxy_time,proxy_life,proxy_web_check_time]
    proxy_info_all.append(proxy_info)
print(proxy_info_all)
opener.close()
proxy_to_db(proxy_info_all)  

In [ ]:
proxy_db = MySQLdb.connect("nfs.flashield.com", "NTTask", "NTTask123!@#", "NTTask", charset='utf8')

In [ ]:
proxy_db.close()